<a href="https://colab.research.google.com/github/nceder/qpb4e/blob/main/code/Chapter%2017/Chapter_17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 17 Data types as objects

# 17.1 Types are objects, too

In [1]:
type(5)

int

In [2]:
type(['hello', 'goodbye'])

list

In [3]:
type_result = type(5)
type(type_result)

type

# 17.2 Using types

In [4]:
type("Hello") == type("Goodbye")

True

In [5]:
type("Hello") == type(5)

False

# 17.3 Types and user-defined classes

In [6]:
class A:
    pass

class B(A):
    pass


In [7]:
b = B()
type(b)

__main__.B

In [8]:
b.__class__

__main__.B

In [9]:
b_class = b.__class__
b_class == B

True

In [10]:
b_class.__name__

'B'

In [11]:
b_class.__bases__

(__main__.A,)

In [12]:
class C:
    pass

class D:
    pass

class E(D):
    pass

x = 12
c = C()
d = D()
e = E()
isinstance(x, E)

False

In [13]:
isinstance(c, E)             #A

False

In [14]:
isinstance(e, E)

True

In [15]:
isinstance(e, D)             #B

True

In [16]:
isinstance(d, E)                #C

False

In [17]:
y = 12
isinstance(y, type(5))             #D

True

In [18]:
issubclass(C, D)

False

In [19]:
issubclass(E, D)

True

In [20]:
issubclass(D, D)           #E

True

In [21]:
issubclass(e.__class__, D)

True

### Quick Check: Types
Suppose that you want to make sure that object `x` is a `list` before you try appending to it. What code would you use? What would be the difference between using `type()` and `isinstance()`? Would this be the look before you leap (LBYL) or easier to ask forgiveness than permission (EAFP) of programming? What other options might you have besides checking the type explicitly?

#### Solution
```python
x = []
if isinstance(x, list):
    print("is list")
    ```
Using type would get only lists, not anything that subclasses lists. Either way, it's LBYL programming.
You might also wrap the append in a try... except block and catch TypeError exceptions, which would be more EAFP.


# 17.5 What is a special method attribute?

### Listing 17.1 File color_module.py

In [22]:
# Listing 17.1 File color_module.py

class Color:
    def __init__(self, red, green, blue):
        self._red = red
        self._green = green
        self._blue = blue
    def __str__(self):
        return f"Color: R={self._red:d}, G={self._green:d}, B={self._blue:d}"

In [23]:
#from color_module import Color
c = Color(15, 35, 3)

In [24]:
print(c)

Color: R=15, G=35, B=3


# 17.7 The `__getitem__` special method attribute

### Quick Check: __getitem__ The example use of __getitem__ above is very limited and won’t work correctly in many situations. What are some cases in which the implementation above will fail or work incorrectly?

#### Solution
This implementation will not work if you try to access an item directly by index; neither can you move backward.

In [25]:
class LineReader:
    def __init__(self, filename):
        self.fileobject = open(filename, 'r')              #A
    def __getitem__(self, index):
        line = self.fileobject.readline()                  #B
        if line == "":                                #C
            self.fileobject.close()      #D
            raise IndexError         #E

        else:
            return line.split("::")[:2]                    #F


# 17.8 Giving an object full list capability

In [26]:
class TypedList:
    def __init__(self, example_element, initial_list=[]):
        self.type = type(example_element)                     #A
        if not isinstance(initial_list, list):
            raise TypeError("Second argument of TypedList must "
                          "be a list.")
        for element in initial_list:
                if not isinstance(element, self.type):
                    raise TypeError("Attempted to add an element of "
                                  "incorrect type to a typed list.")
        self.elements = initial_list[:]

In [27]:
class TypedList:
    def __init__(self, example_element, initial_list=[]):
        self.type = type(example_element)
        if not isinstance(initial_list, list):
            raise TypeError("Second argument of TypedList must "
                            "be a list.")
        for element in initial_list:
            self.__check(element)
        self.elements = initial_list[:]
    def __check(self, element):
        if type(element) != self.type:
            raise TypeError("Attempted to add an element of "
                            "incorrect type to a typed list.")
    def __setitem__(self, i, element):
        self.__check(element)
        self.elements[i] = element
    def __getitem__(self, i):
        return self.elements[i]

In [28]:
x = TypedList("", 5 * [""])
x[2] = "Hello"
x[3] = "There"
print(x[2] + ' ' + x[3])

Hello There


In [29]:
a, b, c, d, e = x
a, b, c, d

('', '', 'Hello', 'There')

### Try This: Implementing list special methods
Try implementing the `__len__` and `__delitem__` special methods for TypedList, as well as an append method.

In [ ]:
# @title
class TypedList:
    def __init__(self, example_element, initial_list=[]):
        self.type = type(example_element)
        if not isinstance(initial_list, list):
            raise TypeError("Second argument of TypedList must "
                            "be a list.")
        for element in initial_list:
            self.__check(element)
        self.elements = initial_list[:]
    def __check(self, element):
        if type(element) != self.type:
            raise TypeError("Attempted to add an element of "
                            "incorrect type to a typed list.")
    def __setitem__(self, i, element):
        self.__check(element)
        self.elements[i] = element
    def __getitem__(self, i):
        return self.elements[i]

    ################################
    ## SOLUTION
    ## added methods
    def __delitem__(self, i):
        del self.elements[i]
    def __len__(self):
        return len(self.elements)
    def append(self, element):
        self.__check(element)
        self.elements.append(element)

x = TypedList(1, [1,2,3])
print(len(x))
x.append(1)
del x[2]


## 17.9.1 Subclassing list

In [30]:
class TypedListList(list):
    def __init__(self, example_element, initial_list=[]):
        self.type = type(example_element)
        if not isinstance(initial_list, list):
            raise TypeError("Second argument of TypedList must "
                            "be a list.")
        for element in initial_list:
            self.__check(element)
        super().__init__(initial_list)

    def __check(self, element):
        if type(element) != self.type:
            raise TypeError("Attempted to add an element of "
                            "incorrect type to a typed list.")

    def __setitem__(self, i, element):
        self.__check(element)
        super().__setitem__(i, element)

In [31]:
x = TypedListList("", 5 * [""])
x[2] = "Hello"
x[3] = "There"
print(x[2] + ' ' + x[3])

Hello There


In [32]:
a, b, c, d, e = x
a, b, c, d

('', '', 'Hello', 'There')

In [33]:
x[:]

['', '', 'Hello', 'There', '']

In [34]:
del x[2]
x[:]

['', '', 'There', '']

In [35]:
x.sort()
x[:]

['', '', '', 'There']

## 17.9.2 Subclassing UserList

In [36]:
from collections import UserList
class TypedUserList(UserList):
    def __init__(self, example_element, initial_list=[]):
        self.type = type(example_element)
        if not isinstance(initial_list, list):
            raise TypeError("Second argument of TypedList must "
                            "be a list.")
        for element in initial_list:
            self.__check(element)
        super().__init__(initial_list)

    def __check(self, element):
        if type(element) != self.type:
            raise TypeError("Attempted to add an element of "
                            "incorrect type to a typed list.")
    def __setitem__(self, i, element):
        self.__check(element)
        self.data[i] = element
    def __getitem__(self, i):
        return self.data[i]

In [37]:
x = TypedUserList("", 5 * [""])
x[2] = "Hello"
x[3] = "There"
print(x[2] + ' ' + x[3])

Hello There


In [38]:
a, b, c, d, e = x
a, b, c, d

('', '', 'Hello', 'There')

In [39]:
x[:]

['', '', 'Hello', 'There', '']

In [40]:
del x[2]
x[:]

['', '', 'There', '']

In [41]:
x.sort()
x[:]

['', '', '', 'There']

### Quick Check: Special method attributes and subclassing  types

Suppose that you want a dictionary-like type that allows only strings as keys (maybe to make it work like a shelf object, as described in chapter 13). What options would you have for creating such a class? What would be the advantages and disadvantages of each option?

#### Solution

You could use the same approach as you did for `TypedList` and inherit from the `UserDict` class. You could also inherit directly from `dict`, or you could implement all of the `dict` functionality yourself.

Implementing everything yourself provides the most control but is the most work and most prone to bugs. If the changes you need to make are small (in this case, just checking the type before adding a key), it might make the most sense to inherit directly from `dict`. On the other hand, inheriting from `UserDict` is probably safest, because the internal `dict` object will continue to be a regular `dict`, which is a highly optimized and mature implementation.

# Lab 17: Creating a string only key:value dictionary

The quick check above mentions creating a dictionary that only allows strings as keys. Let's that idea a step further and actually implement a dictionary that only allows strings for both the keys and values. This sort of dictionary might be useful for example to cache URL's and web pages in a web application.

As mentioned in discussing lists above, you would have three possible approaches - write a class from scratch, inherit from the built-in dictionary, or inherit from UserDictionary. I would suggest for the best combination of simplicity and functionality that you inherit from the built-in `dict` type and override the  `__setitem__()` method.  

In [ ]:
""" Create a dictionary that allows only strings for keys and values"""

class StringDict(dict):
    def __setitem__(self, key, value):
        if not isinstance(key, str):
            raise TypeError('keys must be strings') #A
        if not isinstance(value, str):
            raise TypeError('values must be strings')
        super().__setitem__(key, value)

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        key_error = any(not isinstance(_, str) for _ in self.keys()) #B
        if key_error:
            raise TypeError('keys must be strings')
        value_error = any(not isinstance(_, str) for _ in self.values())
        if value_error:
            raise TypeError('values must be strings')

### Tests

These can be run after executing each cell which declares StringDict class.

In [ ]:
test_dict = StringDict()
test_dict['a'] = 'b'
test_dict['c'] = 'd'
print(test_dict)  # Output: {'a': 'b', 'c': 'd'}

test_dict = StringDict(a='b', c='d')
print(test_dict)  # Output: {'a': 'b', 'c': 'd'}

test_dict = StringDict([('a', 'b'), ('c', 'd')])
print(test_dict)  # Output: {'a': 'b', 'c': 'd'}


{'a': 'b', 'c': 'd'}
{'a': 'b', 'c': 'd'}
{'a': 'b', 'c': 'd'}


In [ ]:
test_dict = StringDict()
test_dict['a'] = 1  # Raises TypeError: values must be strings
test_dict[2] = 'd'  # Raises TypeError: keys must be strings

test_dict = StringDict({a:1})  # Raises TypeError: values must be strings
test_dict = StringDict({2:'d'})  # Raises TypeError: keys must be strings

test_dict = StringDict([(1, 'b'), ('c', 'd')])  # Raises TypeError: keys must be strings
test_dict = StringDict([('a', 1), ('c', 'd')])  # Raises TypeError: values must be strings

TypeError: values must be strings

### Copilot solution

Prompt:

```Create a subclass of dictionary that allows only strings as both keys and values.```

In [ ]:
class StringDict(dict):
    def __setitem__(self, key, value):
        if not isinstance(key, str):
            raise TypeError('keys must be strings')
        if not isinstance(value, str):
            raise TypeError('values must be strings')
        super().__setitem__(key, value)

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        for key, value in self.items():
            if not isinstance(key, str):
                raise TypeError('keys must be strings')
            if not isinstance(value, str):
                raise TypeError('values must be strings')

### Colaboratory solution

Prompt 1:

```Create a subclass of dictionary that allows only strings as both keys and values.```

In [ ]:
class StringDict(dict):
  def __setitem__(self, key, value):
    if not isinstance(key, str) or not isinstance(value, str):
      raise TypeError("Both keys and values must be strings.")
    super().__setitem__(key, value)

Prompt 1:

```Create a subclass of dictionary that allows only strings as both keys and values, including when creating the instance.```

In [ ]:
class StringDict(dict):
  def __init__(self, *args, **kwargs):
    super().__init__()
    for key, value in dict(*args, **kwargs).items():   #A
      self[key] = value      #B

  def __setitem__(self, key, value):
    if not isinstance(key, str) or not isinstance(value, str):
      raise TypeError("Both keys and values must be strings.")
    super().__setitem__(key, value)